In [2]:
import numpy as np
import pandas as pd
import os
import load
import multiprocessing as mul
from multiprocessing import Process

## IMPORTING AND SORTING THE DATA

WE HAVE CHOSEN ICECUBE 2008-18 DATA FOR THIS STUDY


THE ICE CUBE DATA SET HAS 1134450 NEUTRINO EVENTS


We select neutrino events with Energy >= 100TeV = 10^5 GeV

i.e log10(E/GeV) > 5


There are 192107 such neutrino events in this data

The ms pulsars are taken from the ATNF Catalogue

There are 441 pulsars (as of May 2022 when the study started)

All the pulsars lie in the declination range of -87 to +87 degrees

In [3]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]

icdata = icdata.sort_values('log10(E/GeV)')
icdata = icdata.reset_index()
icdata = icdata.drop('index', axis=1)
icdata2 = icdata[icdata['log10(E/GeV)'] > 5]
icdata2 = icdata2.reset_index()
icdata2 = icdata2.drop('index', axis=1)
icdata2

#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/10milsecpsr.txt", 'r')
lines = f.readlines()

content=[]
column=lines[3].split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
mspdata = pd.DataFrame(content).drop(range(0,6)).dropna().drop([2,6,8,10,11,13,14], axis=1)
f.close()
line = []
lines = []

mspdata.columns = column
column = []
content=[]
mspdata.dropna(inplace=True)
mspdata = mspdata.reset_index()
mspdata = mspdata.drop(["index", "#"], axis=1)

In [4]:

msra = [float(i) for i in mspdata['RAJD']]
msdec = [float(i) for i in mspdata['DECJD']]
extensions = 441 - len(icdata2)%441
icra = [float(i) for i in icdata2['RA[deg]']]
icdec = [float(i) for i in icdata2['Dec[deg]']]
icra.extend([0]*extensions)
icdec.extend([0]*extensions)

Selecting events which are at an angular distance < $20^\circ$ from a particular ms psr source = $N$

In [5]:
def S_i(i, s, x, xs, sg):
    return np.exp((x - xs[s])**2 / (2 * (sg[i])**2)) / (2 * np.pi * (sg[i] ** 2))

$B_i$ = No.of Events in a dataset/Solid angle average acrpss a band of +-6 deg in declination around the $i^{th}$ source

Solid angle span = $2\pi\cos(\cos(\delta + 3) - cos(\delta - 3))$

We estimate the background PDF by calculating the ratio of average number of events per steradian at each
declination and the total number of events. 

In [6]:
def b_ivec(lon1, lat1, lon2, lat2):

    #Convert decimal degrees to Radians:
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)


    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    return (np.rad2deg(np.arccos(a)), a)

In [10]:
B = []
l_ic = len(icdata2)
for i in range(len(mspdata)):
#returns $B_i$ as defined above
#def B_i(i):

    r = []
    psra = float(mspdata['RAJD'][i])
    psdec = float(mspdata['DECJD'][i])
    pssa = np.cos(np.cos(np.deg2rad(psdec) + np.cos(np.deg2rad(3))) - np.cos(np.deg2rad(psdec) - np.cos(np.deg2rad(3))))
    for j in range(l_ic):
        if icdec[j] > np.deg2rad(psdec) - pssa and icdec[j] < np.deg2rad(psdec) + pssa:
            r.append(j)
    B.append([i,r])

    

In [12]:
B[0]

[0,
 [7677,
  10754,
  12348,
  13195,
  20126,
  23695,
  25087,
  35553,
  35612,
  41128,
  46566,
  47024,
  47574,
  56511,
  56530,
  58542,
  94046,
  102327,
  114574,
  124081,
  138373,
  165675,
  166100,
  187073]]

n_s = Signal Events from point source

In [ ]:
5 in range(4,10)

In [ ]:
def P_i(i,s, S, B, n, N):
    return n[s] * S[i] / N + (1 - n[s]/N)*B[i]


In [ ]:
def L_i(N, i, P, s):
    L = 1
    for j in range(N):
        L *= P_i(j,s,S,B,n,N)
    return L

In [ ]:
def TS(ns, L):
    return 2 * np.log10(L_i())